In [ ]:
from collections import deque

In [ ]:
with open('../inputs/23.txt') as f:
    data = f.read().splitlines()

In [ ]:
def get_neighbours(loc, prev_dir, forest, use_slopes):
    r, c = loc
    neighbours = []
    slopes = 'v<^>'
    
    for i, (dr, dc) in enumerate([(1, 0), (0, -1), (-1, 0), (0, 1)]):
        if (dr * -1, dc * -1) == prev_dir:
            continue
        
        if use_slopes and forest[r][c] in slopes and slopes[i] != forest[r][c]:
            continue

        nr, nc = (r + dr, c + dc)
        
        if not (0 <= nr < len(forest)) or not (0 <= nc < len(forest[0])):
            continue
        
        if forest[nr][nc] == '#':
            continue
        
        neighbours.append(((nr, nc), (dr, dc)))
        
    return neighbours

In [ ]:
def build_graph(start, end, forest, use_slopes):
    queue = deque([(start, start, 0, None)])
    graph = {}
    visited = set()

    while queue:
        loc, prev_node, steps, prev_dir = queue.popleft()
        
        if loc == end:
            if prev_node not in graph:
                graph[prev_node] = {}
            graph[prev_node][loc] = steps
            continue
        
        if (loc, prev_node) in visited:
            continue
        visited.add((loc, prev_node))

        neighbours = get_neighbours(loc, prev_dir, forest, use_slopes)
        is_junction = len(neighbours) > 1
        
        if is_junction or loc == end:
            if prev_node not in graph:
                graph[prev_node] = {}
            graph[prev_node][loc] = steps
            
            for neighbour, direction in neighbours:
                queue.append((neighbour, loc, 1, direction))
        else:
            if neighbours:
                neighbour, direction = neighbours[0]
                queue.append((neighbour, prev_node, steps + 1, direction))
    
    if use_slopes:     
        for node in graph.keys():
            for k, v in graph.items():
                for distance, n in v:
                    if n == node:
                        graph[node].add((distance, k))
    
    return graph

In [ ]:
def dfs(graph, current, visited=None):
    if visited is None:
        visited = set()

    visited.add(current)

    max_length = 0
    if current in graph:
        for neighbor in graph[current]:
            weight = graph[current][neighbor]
            if neighbor not in visited:
                current_length = weight + dfs(graph, neighbor, visited.copy())
                max_length = max(max_length, current_length)

    return max_length

In [ ]:
def find_longest_path(forest, use_slopes = True):
    start = (0, forest[0].index('.'))
    end = (len(forest) - 1, forest[len(forest) - 1].index('.'))
    graph = build_graph(start, end, forest, use_slopes)
    
    return dfs(graph, start)

In [ ]:
# Part 1
find_longest_path(data)

In [ ]:
# Part 2
find_longest_path(data, False)